In [1]:
%run ./VA_sentencing_reform_v1_utils.ipynb


52291


In [2]:
reload(incarceration_transitions)
reload(population_simulation)

<module 'spark.sentencing_policy_impact_v1.population_simulation' from '../../spark/sentencing_policy_impact_v1/population_simulation.py'>

In [3]:
from spark.E2E_v2.super_simulation import SuperSimulation
from spark.E2E_v2.incarceration_transitions import IncarceratedTransitions
from spark.E2E_v2.release_transitions import ReleasedTransitions


In [12]:
years = range(2020, 2025)
deferred_pop = [210,280,350,420,490]
#deferred_pop = [100,100,100,100,100]

outflows_data = pd.DataFrame(data = {'year':years,
                                     'total_population':deferred_pop
                                    }
                            )
outflows_data['subgroup'] = 'affected_population'
outflows_data['compartment'] = 'pretrial' 
outflows_data['outflow_to'] = 'supervision'
outflows_data

,year,total_population,subgroup,compartment,outflow_to
0,2020,100,affected_population,pretrial,supervision
1,2021,100,affected_population,pretrial,supervision
2,2022,100,affected_population,pretrial,supervision
3,2023,100,affected_population,pretrial,supervision
4,2024,100,affected_population,pretrial,supervision


In [13]:
transition_data = pd.DataFrame({
    'compartment': ['supervision', 'supervision', 'supervision', 'release', 'prison'],
    'outflow_to': ['release', 'prison', 'prison', 'release', 'prison'],
    'total_population': [77,11,12,1,1],
    'compartment_duration': [2,1,2,1,1]
})
transition_data['subgroup'] = 'affected_population'
transition_data


,compartment,outflow_to,total_population,compartment_duration,subgroup
0,supervision,release,77,2,affected_population
1,supervision,prison,11,1,affected_population
2,supervision,prison,12,2,affected_population
3,release,release,1,1,affected_population
4,prison,prison,1,1,affected_population


In [18]:
user_inputs = {'projection_years': 5, 'policy_year': 2020, 'start_year': 2020}
compartments_architecture = {'prison': IncarceratedTransitions, 'release': ReleasedTransitions, 
                             'prison_shell': None, 'supervision': IncarceratedTransitions}

compartments_architecture = {'prison': IncarceratedTransitions,
                             'release': ReleasedTransitions,
                             'supervision': IncarceratedTransitions,
                             'pretrial': None}



disaggregation_axes = ['subgroup']
policy_projection = SuperSimulation()
policy_projection.initialize_data(outflows_data, transition_data, compartments_architecture, disaggregation_axes)
policy_projection.set_user_inputs(user_inputs)
projection_results = policy_projection.simulate_baseline(output_compartment = 'prison', outflow_to = 'release', initialization_period = 0)

initialization time:  0.006174802780151367
simulation_time:  0.1240699291229248


../../spark/E2E_v2/compartment_transitions.py:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  self.historical_outflows[transitions_below_threshold_slice['total_population']]


KeyError: 'release'

In [19]:
results = policy_projection.pop_simulations['baseline'].population_projections
results

,total_population,compartment,year,simulation_group
2020,0.0,prison,2020,affected_population
2021,11.0,prison,2021,affected_population
2022,34.0,prison,2022,affected_population
2023,57.0,prison,2023,affected_population
2024,80.0,prison,2024,affected_population
2020,0.0,release,2020,affected_population
2021,0.0,release,2021,affected_population
2022,77.0,release,2022,affected_population
2023,154.0,release,2023,affected_population
2024,231.0,release,2024,affected_population
